In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import re
import nltk
import string

from nltk.corpus import wordnet
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [28]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\z\AppData\Roaming\nltk_data...


True

In [20]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [22]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\z\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

Import CSV

In [836]:
#read csv file
df = pd.read_csv('data/Timeline/Yearn (Yearn Finance).csv')
df

,Unnamed: 0,Date,screen_name,username,tweet_content,retweets,favourites
0,771,2020-02-03 22:55:17,yearn.finance,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6
1,770,2020-02-04 04:51:37,yearn.finance,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30
2,769,2020-02-05 00:50:46,yearn.finance,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8
3,768,2020-02-05 20:33:37,yearn.finance,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6
4,767,2020-02-05 20:52:13,yearn.finance,iearnfinance,Switch to the best savings account in the worl...,3,12
...,...,...,...,...,...,...,...
767,4,2021-06-13 02:09:13,yearn.finance,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463
768,3,2021-06-13 22:02:33,yearn.finance,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161
769,2,2021-06-15 19:03:01,yearn.finance,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97
770,1,2021-06-16 19:34:51,yearn.finance,iearnfinance,We are excited to announce the launch of our n...,281,620


Drop Unnecessary Column

In [837]:
df = df.drop(['Unnamed: 0','screen_name'], axis=1)

In [838]:
df.head()

,Date,username,tweet_content,retweets,favourites
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12


Check Info

In [839]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 5 columns):
Date             772 non-null object
username         772 non-null object
tweet_content    772 non-null object
retweets         772 non-null int64
favourites       772 non-null int64
dtypes: int64(2), object(3)
memory usage: 30.3+ KB


Remove Duplicates Row (For Text)

In [840]:
df[['username','tweet_content']].duplicated().sum()

0

In [841]:
df = df.drop_duplicates(subset=['username','tweet_content'])
df.reset_index(drop=True, inplace=True)

In [842]:
df

,Date,username,tweet_content,retweets,favourites
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12
...,...,...,...,...,...
767,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463
768,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161
769,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97
770,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620


In [843]:
for data in df['tweet_content']:
    print(data)

https://t.co/c4WRWeU8tt for yields and stuff. I really don't know how to twitter...
https://t.co/BAcrlUTqrQ updated;

Live on-chain rates from;

@UniswapExchange 
@compoundfinance 
@bzxHQ 
@AaveAave 
@ddexio2018 
@LendfMe 

Including real-time on-chain APR calculator for your investment sum for;

$DAI, $TUSD, $USDC, $USDT, $ETH and more. https://t.co/SdO6LvfoFR
Daily APR https://t.co/BAcrlUTqrQ

$DAI 8.75% dominated by @bzxHQ. Static since they introduced 9% fixed rate, they own it.

@USDC  ~4% @compoundfinance and @dydxprotocol fighting over dominance.

$USDT comfy at 20.75% at @AaveAave (insane)

@CurveFinance $DAI strong at 9.11% https://t.co/dtadzP8Wx2
Daily APR &amp; (APY) @iearnfinance 

$DAI 18.29% (8.41%) @dydxprotocol 
$USDC 6.78% (3.93%) @AaveAave 
$USDT 32% (11.76%) @AaveAave ~ they are dominating $USDT
DAI-USDC 9.13% (9.31%) @CurveFinance 

https://t.co/BAcrlUTqrQ https://t.co/Oo9yXpApzv
Switch to the best savings account in the world with zero effort, completely free. 

Th

A promising new levered $DAI strategy to farm $COMP on @compoundfinance is currently being tested. 

This strategy was proposed by a community member, and we encourage individuals to propose ideas for new strategies on our governance forum. 

https://t.co/8zce0Kwf3l https://t.co/tprmObQttc
Weekly governance roundup by Yearn Communications member @yfi_nomad.

Be sure to read and subscribe to stay up to date with Yearn governance proposals. https://t.co/k2vku8hXwC
We are hosting our second Ask Me Anything on Discord held next Tuesday, October 27th at 16:00 UTC. 

You can also ask us a question on this google form. 

https://t.co/5apkcjWAN4

Or ask during the live AMA on discord. 

https://t.co/bu3nS7hTN4
Weekly State of the Vaults is out!

https://t.co/KMj0k52Cnh
We would like to remind everyone that we are hosting our 2nd AMA this Tuesday, October 27th at 16:00 UTC on our discord channel. 

You can also submit one question using this google form. 

https://t.co/5apkcjWAN4
Our latest wee

Remove Links, Hashtags, username etc

In [844]:
cleanedText = []
linkRegex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
usernameRegex = r'@[^\s]+'
hashtagRegex = r'#[^\s]+'
hash_addressRegex = r'0x[^\s]+'

for row in df['tweet_content'].iteritems():
    x = row[-1]
    x = re.sub(linkRegex, '', x)
    x = re.sub(usernameRegex, '', x)
    x = re.sub(hashtagRegex, '', x)
    x = re.sub(hash_addressRegex, '', x)
    x = x.strip()
    cleanedText.append(x)

In [845]:
df["cleanedText"] = cleanedText 

In [846]:
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yields and stuff. I really don't know how ...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated;\n\nLive on-chain rates from;\n\n \n \...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,Daily APR \n\n$DAI 8.75% dominated by Static ...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,Daily APR &amp; (APY) \n\n$DAI 18.29% (8.41%)...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,Switch to the best savings account in the worl...
...,...,...,...,...,...,...
767,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,"What if Yearn = the web's ""yield function""?\n\..."
768,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,We are thrilled to announce that Yearn has jus...
769,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,Read this week's Yearn newsletter 📰\n\n💠 Yearn...
770,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,We are excited to announce the launch of our n...


In [847]:
for data in df["cleanedText"]:
    print(data)

for yields and stuff. I really don't know how to twitter...
updated;

Live on-chain rates from;

 
 
 
 
 
 

Including real-time on-chain APR calculator for your investment sum for;

$DAI, $TUSD, $USDC, $USDT, $ETH and more.
Daily APR 

$DAI 8.75% dominated by  Static since they introduced 9% fixed rate, they own it.

  ~4%  and  fighting over dominance.

$USDT comfy at 20.75% at  (insane)

 $DAI strong at 9.11%
Daily APR &amp; (APY)  

$DAI 18.29% (8.41%)  
$USDC 6.78% (3.93%)  
$USDT 32% (11.76%)  ~ they are dominating $USDT
DAI-USDC 9.13% (9.31%)
Switch to the best savings account in the world with zero effort, completely free. 

Thank you
updated

Support for;

 
 
 
 
 
 
 

 

Wallet connected now displayed in top right corner.
updated

Now includes APY (realized) profits from uniswap year-to-date, and  year-to-date adjusted.

 APY to-date adjusted to annualized:

$DAI 9.38%
$USDC 7.25%
$USDT 13.96%
$SUSD 13.54%
DAI-USDT 14.84%
updated for all the mobile users

All pages optimiz

Yearn v2 Vaults are under active development.

Will include new vaults and strategies.

We are very excited for this. 

Additional details will be shared when available.
8m DAI related to the $EMN incident was sent to the Yearn multi-sig address. 

This amount is being distributed to affected addresses based on a snapshot taken at block 10954410

You claim your DAI here:
Members of the Yearn developer team will be featured on  discussing yVaults and how to build new strategies on Yearn. 

Watch live tomorrow 10/1 at 1pm ET.
We are launching a competition to select artwork to be the new face of yInsure.

Submit 2 images. One image for all insurance under 100 ETH, and another image for insurance over 100 ETH. 

Submissions are open starting October 15th and end October 17th (48 hours).
Stay updated on Yearn Governance with this weekly roundup by
New proposal to create additional yVaults that would farm $UNI. 

Join the discussion.
Vaults v2 are coming. Stay tuned.
The latest Yearn Financ

In [848]:
replace_dict =  {
"ain't": "am not",
"ain’t": "am not",
"aren’t": "are not",
"aren't": "are not",
"can't": "cannot",
"can’t": "cannot",
"Can't":"cannot",
"can't've": "cannot have",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn’t": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"didn’t": "did not",
"don’t": "do not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"how’s": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i’ll": "I will",
"i'll": "I will",
"i'll've": "I will have",
"i'll": "i will",
"i'm": "I am",
"im": "I am",
"i’m": "I am",
"i've": "I have",
"i’ve": "I have",  
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it’ll": "it will",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"it’s": "it is",
"It’s": "it is",
"It's": "it is",
"let's": "let us",
"irl":"in real life",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"That's": "that is",
"that’s": "that is",
"there'd": "there had",
"There’s": "there is",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"we’re": "we are",
"we’ve": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y’all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"here's": "here is",
"what’s": "what is",
"m00n":"moon",
"gt": "got",
"cuz": "because",
"coz":"because",
"bc": "because",
"rn": "right now",
"pls": "please",
"alil": "a little",
"q's": "q",
"luv": "love",
"fwiw": "for what it is worth",
"francξ": "france",
"futurξ": "future",
"financξ" : "finance",
"founder's": "founder",
"secs": "seconds",
"r.i.p." : "rest in peace",
"tyvm": "thank you very much",
"u": 'you',
"kindof" : "kind of"
}

In [849]:
cleanedText = []
for row in df['cleanedText'].iteritems():
    tweet = row[-1]
    tweet = tweet.lower() #change to lower case
    tweet = " ".join(tweet.split()) #remove extra spaces in string
    test = tweet.split() #get indiv words to replace
    tweet = [replace_dict.get(w, w) for w in test] #replace common short forms
    tweet = " ".join(tweet)

    cleanedText.append(tweet)

In [850]:
df['cleanedText'] = cleanedText
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yields and stuff. i really do not know how...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated; live on-chain rates from; including r...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,daily apr $dai 8.75% dominated by static since...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,daily apr &amp; (apy) $dai 18.29% (8.41%) $usd...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,switch to the best savings account in the worl...
...,...,...,...,...,...,...
767,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,"what if yearn = the web's ""yield function""? in..."
768,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,we are thrilled to announce that yearn has jus...
769,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,read this week's yearn newsletter 📰 💠 yearn tv...
770,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,we are excited to announce the launch of our n...


In [851]:
for text in df['cleanedText']:
    print(text)

for yields and stuff. i really do not know how to twitter...
updated; live on-chain rates from; including real-time on-chain apr calculator for your investment sum for; $dai, $tusd, $usdc, $usdt, $eth and more.
daily apr $dai 8.75% dominated by static since they introduced 9% fixed rate, they own it. ~4% and fighting over dominance. $usdt comfy at 20.75% at (insane) $dai strong at 9.11%
daily apr &amp; (apy) $dai 18.29% (8.41%) $usdc 6.78% (3.93%) $usdt 32% (11.76%) ~ they are dominating $usdt dai-usdc 9.13% (9.31%)
switch to the best savings account in the world with zero effort, completely free. thank you
updated support for; wallet connected now displayed in top right corner.
updated now includes apy (realized) profits from uniswap year-to-date, and year-to-date adjusted. apy to-date adjusted to annualized: $dai 9.38% $usdc 7.25% $usdt 13.96% $susd 13.54% dai-usdt 14.84%
updated for all the mobile users all pages optimized for mobile friendly access.
daily apr (apy) update $dai 9.42

no clue who made this, anyone know? very cool

2nd reward emission was added. ~$12,129.192
updated pool and cover disabled dead links to iearn updated to yearn updates on yleverage and iborrow to follow
the token is to manage the platform. if you are not interested in managing the platform, stay away from the token.
4 new yield strategies coming soon, all have outperformed yearn base layer. select your risk appetite and farm from safe (deposits only) to degenerate (maxed out leverage)
with next release we will also enable credit delegation. allowing you to farm yield with; $susd, $busd, $eth, $lend, $ren, $knc, $link, $mana, $mkr, $snx without loosing any exposure to the asset itself.
only a few blocks remaining
🥳
the show goes on. very excited to see what emission schedule the community votes on. while you all manage, we will continue to ship. want to contribute to the future of yfi and the ecosystem?
proposal "for" 61%, quorum 64%. distribution will continue. exact numbers to be vote

mip10c3-sp10 proposal: yfiusd oracle (collateral onboarding oracle assessment)
introducing syntheticrebasedollar a credit based rebase index
updated improved analytics and expanded details view growth as a representation of 1 day, 3 days, 1 week, or inception view detailed breakdown of daily/weekly/yearly growth and current strategies view overall system stats, deployed strategies, and more
updated; removed 1 day apy values. from feedback received it was clear it was confusing users. with current gas prices some yield only occur &gt; 24 hours. so any given time slice might show 0% we recommend monitoring &gt; 1 month.
we are currently processing refunds related to $emn. more information can be found in the thread below. an update will be given in the near future when refunds are sent out.
the yyfi vault temporarily (blocks 10923319-10954777) had an excess fee of 5% per withdrawal, instead of 0.5% per withdrawal. the excess fee amount has been distributed to the 23 users affected. we ap

in this week's state of the vaults 🏦 💠 zap into yvaults with 💠 tvl for v2 vaults is exploding 💠 yfi vault gets a new strategy utilizing and more!
yearn vault top performers 🏆 stablecoin pool apys: 🥇 dusd: 39.96% 🥈 usdp: 37.89% 🥉 usdn: 35.63% btc and eth apys: 🥇 obtc: 28.22% 🥈 pbtc: 27.15% 🥉 steth: 25.52% all other asset apys: 🥇 yfi: 37.30% 🥈 crvlink: 21.15% find them all here:
earn 108% apy with our new iron bank yvault! 🚀 we are thrilled to launch a new yvault utilizing the iron bank liquidity pool on 😍 zap into this vault via our integration with ⚡️ stay tuned for more integrations within our ecosystem 🍦
be sure to read this week's newsletter 🤓 💠 1-click zap into yearn vaults now live 💠 yearn total value locked is exploding 💠 yla/usdc pool live on sushiswap 💠 new strategy for the yfi vault and more inside! 🚀
we have upgraded our snapshot space you can now vote with yfi in your wallet, yvyfi, bancor, balancer, uniswap, sushiswap, makerdao and unit. a reminder to withdraw your yfi from

In [852]:
lemmatizer = nltk.WordNetLemmatizer()

In [853]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [854]:
cleanedText = []
for row in df['cleanedText'].iteritems():
    tweet = row[-1]
    tweet = re.sub('[^a-zA-Z]', ' ', tweet) #remove punctuation
    tweet = tweet.lower() #change to lower case
    tweet = " ".join(tweet.split()) #remove extra spaces in string
    
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(tweet)) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    tweet = " ".join(lemmatized_sentence)

    cleanedText.append(tweet)

In [855]:
df['cleanedText'] = cleanedText

In [856]:
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yield and stuff i really do not know how t...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated live on chain rate from include real t...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,daily apr dai dominate by static since they in...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,daily apr amp apy dai usdc usdt they be domina...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,switch to the best saving account in the world...
...,...,...,...,...,...,...
767,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,what if yearn the web s yield function in prog...
768,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,we be thrill to announce that yearn have just ...
769,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,read this week s yearn newsletter yearn tvl ha...
770,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,we be excite to announce the launch of our new...


In [857]:
for text in df['cleanedText']:
    print(text)

for yield and stuff i really do not know how to twitter
updated live on chain rate from include real time on chain apr calculator for your investment sum for dai tusd usdc usdt eth and more
daily apr dai dominate by static since they introduce fix rate they own it and fight over dominance usdt comfy at at insane dai strong at
daily apr amp apy dai usdc usdt they be dominate usdt dai usdc
switch to the best saving account in the world with zero effort completely free thank you
updated support for wallet connect now display in top right corner
update now include apy realize profit from uniswap year to date and year to date adjust apy to date adjust to annualized dai usdc usdt susd dai usdt
update for all the mobile user all page optimize for mobile friendly access
daily apr apy update dai usdc usdt susd dai usdc wbtc end of month volatility have calm down a bit
this ui be so much good than our own can we just send all the user over to you
update add build with provider that make building

we would like to remind everyone that we have a vote regard change the fee structure for v yvaults key change eliminate withdrawal fee management fee on aum performance fee split between strategist amp yearn governance
check out our late state of the vault where we cover current vault strategy recently deploy vault and potential upcoming change
read and subscribe to our weekly newsletter two active vote on snapshot new vecrv backscratcher be live test for several new yvaults audit complete by mixbytes
we be excite to share that have develop and be test a new eth dai vault use our goal be to attract and retain the top mind and developer in defi to create new strategy strategist will be compensate for their effort
yip revamp the performance fee split for yvaults this yip would split the performance fee between yearn governance and strategists this vote be bind and now live on snapshot
we have drop something special for all og yfi farmer thank you for your visionary support credit
yip hav

In [858]:
df[['cleanedText','username']].duplicated().sum()

65

In [859]:
df.drop_duplicates(subset=['cleanedText'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yield and stuff i really do not know how t...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated live on chain rate from include real t...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,daily apr dai dominate by static since they in...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,daily apr amp apy dai usdc usdt they be domina...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,switch to the best saving account in the world...
...,...,...,...,...,...,...
702,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,what if yearn the web s yield function in prog...
703,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,we be thrill to announce that yearn have just ...
704,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,read this week s yearn newsletter yearn tvl ha...
705,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,we be excite to announce the launch of our new...


In [860]:
words = {
"x": "",
"v": "",
"amp": "",
"k" : "",
"l" : "",
"n" : "",
"ve" : "",
"d": "",
"l": "",
's':'',
"t":"",
"g": '',
"m": ''
}

In [861]:
cleanedText = []
for row in df['cleanedText'].iteritems():
    tweet = row[-1]
    tweet = tweet.lower() #change to lower case
    tweet = " ".join(tweet.split()) #remove extra spaces in string
    test = tweet.split() #get indiv words to replace
    tweet = [words.get(w, w) for w in test] #replace common short forms
    tweet = " ".join(tweet)

    cleanedText.append(tweet)

In [862]:
df['cleanedText'] = cleanedText
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yield and stuff i really do not know how t...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated live on chain rate from include real t...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,daily apr dai dominate by static since they in...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,daily apr apy dai usdc usdt they be dominate ...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,switch to the best saving account in the world...
...,...,...,...,...,...,...
702,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,what if yearn the web yield function in progr...
703,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,we be thrill to announce that yearn have just ...
704,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,read this week yearn newsletter yearn tvl hav...
705,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,we be excite to announce the launch of our new...


In [863]:
cleanedText = []
for row in df['cleanedText'].iteritems():
    tweet = row[-1]
    tweet = re.sub('[^a-zA-Z]', ' ', tweet) #remove punctuation
    tweet = tweet.lower() #change to lower case
    tweet = " ".join(tweet.split()) #remove extra spaces in string
    
    
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(tweet)) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    tweet = " ".join(lemmatized_sentence)

    cleanedText.append(tweet)

In [864]:
df['cleanedText'] = cleanedText
df

,Date,username,tweet_content,retweets,favourites,cleanedText
0,2020-02-03 22:55:17,iearnfinance,https://t.co/c4WRWeU8tt for yields and stuff. ...,2,6,for yield and stuff i really do not know how t...
1,2020-02-04 04:51:37,iearnfinance,https://t.co/BAcrlUTqrQ updated;\n\nLive on-ch...,7,30,updated live on chain rate from include real t...
2,2020-02-05 00:50:46,iearnfinance,Daily APR https://t.co/BAcrlUTqrQ\n\n$DAI 8.75...,3,8,daily apr dai dominate by static since they in...
3,2020-02-05 20:33:37,iearnfinance,Daily APR &amp; (APY) @iearnfinance \n\n$DAI 1...,2,6,daily apr apy dai usdc usdt they be dominate u...
4,2020-02-05 20:52:13,iearnfinance,Switch to the best savings account in the worl...,3,12,switch to the best saving account in the world...
...,...,...,...,...,...,...
702,2021-06-13 02:09:13,iearnfinance,"What if Yearn = the web's ""yield function""?\n\...",76,463,what if yearn the web yield function in progra...
703,2021-06-13 22:02:33,iearnfinance,We are thrilled to announce that Yearn has jus...,150,1161,we be thrill to announce that yearn have just ...
704,2021-06-15 19:03:01,iearnfinance,Read this week's Yearn newsletter 📰\n\n💠 Yearn...,18,97,read this week yearn newsletter yearn tvl have...
705,2021-06-16 19:34:51,iearnfinance,We are excited to announce the launch of our n...,281,620,we be excite to announce the launch of our new...


In [865]:
for text in df['cleanedText']:
    print(text)

for yield and stuff i really do not know how to twitter
updated live on chain rate from include real time on chain apr calculator for your investment sum for dai tusd usdc usdt eth and more
daily apr dai dominate by static since they introduce fix rate they own it and fight over dominance usdt comfy at at insane dai strong at
daily apr apy dai usdc usdt they be dominate usdt dai usdc
switch to the best saving account in the world with zero effort completely free thank you
updated support for wallet connect now display in top right corner
update now include apy realize profit from uniswap year to date and year to date adjust apy to date adjust to annualized dai usdc usdt susd dai usdt
update for all the mobile user all page optimize for mobile friendly access
daily apr apy update dai usdc usdt susd dai usdc wbtc end of month volatility have calm down a bit
this ui be so much good than our own can we just send all the user over to you
update add build with provider that make building pos

updated new vault add for yfi disclaimer withdrawal fee do not risk your fund yvault initialize at block will provide apr measurement over next hour
yycrv vault update we have reach out to the team to good understand the relationship of vecrv voting to gauge weight we will propose a strategy that commits of vecrv to increase ycrv gauge weight update on
yyfi vault update yvault have be live for less than hour already worth of yfi buy off the market highly unsustainable apy of currently do not expect this to maintain
treasury update to go before governance distribution continue day
tvl breakdown yusd yusdc ytusd ydai yusdt yyfi yalink yusdcd yearn tvl
treasury update remain till treasury be cap and governance distribution continue vote for treasury distribution and funding on
another wild roi calculator appear daily weekly yearly roi include since strategy change amaze work please add a donation address
community health statistic
tvl update yusd yusdc ytusd ydai yusdt yyfi yalink usdcd y

In [866]:
df.to_csv('data/Timeline/Clean Tweets/Yearn (Yearn Finance)_Cleaned.csv',index = False)